In [ ]:
#USED FOR AIRFLOW
import os
try:
    relative_location
except NameError:
    relative_location = ""

if relative_location != "":
    os.chdir(relative_location)

In [15]:
import requests
import pandas as pd

In [50]:
def create_df_from_json(raw_json: dict) -> pd.DataFrame:
    columns = ['ship_id','trx_id','space','speed','skill','defence','attack','morale','price']
    sold_df = pd.DataFrame(columns=columns)

    for i,entry in enumerate(raw_json['data']['auctions']):
        sold_df.loc[i,'trx_id'] = entry['idx']
        sold_df.loc[i,'ship_id'] = entry['token_id']
        sold_df.loc[i,'space'] = entry['raw_space']
        sold_df.loc[i,'speed'] = entry['raw_speed']
        sold_df.loc[i,'skill'] = entry['raw_skill']
        sold_df.loc[i,'defence'] = entry['raw_defence']
        sold_df.loc[i,'attack'] = entry['raw_attack']
        sold_df.loc[i,'morale'] = entry['raw_morale']
        sold_df.loc[i,'price'] = round(float(entry['buyout_price'])/10e+8, 3)
        
    sold_df = sold_df.iloc[::-1].reset_index(drop=True)
    sold_df = sold_df.astype({'space': 'int64', 'speed': 'int64', 'skill': 'int64', 'defence': 'int64',\
        'attack': 'int64', 'morale': 'int64', 'price': 'float64'})
    return sold_df

In [53]:
n_lines = 100000
url = f"https://api.cryptobay.top/bay/cryptobaygetauctionlist?data=%7B%22object%22%3A%7B%7D%2C%22\
token_type%22%3A1%2C%22state%22%3A%221%22%2C%22\
order_by%22%3A%222%22%2C%22page_num%22%3A0%2C%22page_size%22%3A{n_lines}%7D"

raw_json = requests.get(url).json()
sold_ships_file = "db/extracted/all_sold_ships.csv"

In [54]:
sold_df = create_df_from_json(raw_json)

In [61]:
#sold_df

In [60]:
sold_df.to_csv(sold_ships_file, index=False)